In [1]:
from torch.utils.data import DataLoader
from torchvision import transforms
from progressBar import printProgressBar

from torchgeometry.losses import tversky

import medicalDataLoader
import argparse
from utils import *

from UNet_Base import *
import random
import torch
import pdb

import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score 
from sklearn.metrics import confusion_matrix
from torchmetrics import ConfusionMatrix

In [5]:
import warnings
warnings.filterwarnings("ignore")
class DiceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceLoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):
        # print(inputs.shape())
        # print(targets.shape())
        inputs = F.sigmoid(inputs)

        inputs = inputs.view(-1)
        targets = targets.view(-1)

        intersection = (inputs * targets).sum()
        dice = (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)

        return 1 - dice

In [9]:
def runTraining(modelName,checkpoints=None, n_epochs=0):
    print('-' * 40)
    print('~~~~~~~~  Starting the training... ~~~~~~')
    print('-' * 40)

    ## DEFINE HYPERPARAMETERS (batch_size > 1)
    batch_size = 16
    batch_size_val = 8
    lr = 0.001
    epoch = 10
    start_epoch = 0
    if checkpoints != None :
        batch_size = checkpoints['batch_size']
        batch_size_val = checkpoints['batch_size_val']
        lr = checkpoints['lr']    # Learning Rate
        epoch = n_epochs # Number of epochs
        start_epoch = checkpoints['epoch']
        

    root_dir = './Data/'

    print(' Dataset: {} '.format(root_dir))

    ## DEFINE THE TRANSFORMATIONS TO DO AND THE VARIABLES FOR TRAINING AND VALIDATION
    
    transform = transforms.Compose([
        transforms.ToTensor()
    ])

    mask_transform = transforms.Compose([
        transforms.ToTensor()
    ])

    train_set_full = medicalDataLoader.MedicalImageDataset('train',
                                                      root_dir,
                                                      transform=transform,
                                                      mask_transform=mask_transform,
                                                      augment=False,
                                                      equalize=False)

    train_loader_full = DataLoader(train_set_full,
                              batch_size=batch_size,
                              worker_init_fn=np.random.seed(0),
                              num_workers=0,
                              shuffle=True)
    
    
    


    val_set = medicalDataLoader.MedicalImageDataset('val',
                                                    root_dir,
                                                    transform=transform,
                                                    mask_transform=mask_transform,
                                                    equalize=False)

    val_loader = DataLoader(val_set,
                            batch_size=batch_size_val,
                            worker_init_fn=np.random.seed(0),
                            num_workers=0,
                            shuffle=False)


    ## INITIALIZE YOUR MODEL
    num_classes = 4 # NUMBER OF CLASSES

    print("~~~~~~~~~~~ Creating the UNet model ~~~~~~~~~~")
    modelName = 'Test_Model'
    print(" Model Name: {}".format(modelName))

    ## CREATION OF YOUR MODEL
    net = UNet(num_classes)
    if checkpoints != None and checkpoints['model_state_dict'] != None:
        net.load_state_dict = checkpoints['model_state_dict']
    

    print("Total params: {0:,}".format(sum(p.numel() for p in net.parameters() if p.requires_grad)))

    # DEFINE YOUR OUTPUT COMPONENTS (e.g., SOFTMAX, LOSS FUNCTION, ETC)
    softMax = torch.nn.Softmax()
    CE_loss = torch.nn.CrossEntropyLoss()
    # CE_loss = torch.nn.FocalLoss(weight=net.parameters(), ignore_index=255,
    #                              size_average=True)

    ## PUT EVERYTHING IN GPU RESOURCES    
    if torch.cuda.is_available():
        net.cuda()
        softMax.cuda()
        CE_loss.cuda()

    ## DEFINE YOUR OPTIMIZER
    # optimizer = torch.optim.Adam(net.parameters(), lr=lr)
    optimizer = torch.optim.SGD(net.parameters(), lr=lr, weight_decay=5e-4,momentum=0.9)

    if checkpoints != None and checkpoints['optimizer_state_dict'] != None:
        optimizer.load_state_dict = checkpoints['optimizer_state_dict']

    ### To save statistics ####
    lossTotalTraining = []
    Best_loss_val = 1000
    if checkpoints != None :
        Best_loss_val = checkpoints['val_loss']
    BestEpoch = 0
    
    directory = 'Results/Statistics/' + modelName

    print("~~~~~~~~~~~ Starting the training ~~~~~~~~~~")
    if os.path.exists(directory)==False:
        os.makedirs(directory)

    ## START THE TRAINING
    
    ## FOR EACH EPOCH
    for i in range(start_epoch, epoch):
        net.train()
        lossEpoch = []
        lossValEpoch = []
        mean_acc = np.array([0,0,0,0]).astype(float)
        mean_val_acc = np.array([0,0,0,0]).astype(float)
        DSCEpoch = []
        DSCEpoch_w = []
        num_batches = len(train_loader_full)
        num_batches_val = len(val_loader)
        
        n = 0
        ## FOR EACH BATCH
        for j, data in enumerate(train_loader_full):
            ### Set to zero all the gradients
            net.zero_grad()
            optimizer.zero_grad()

            ## GET IMAGES, LABELS and IMG NAMES
            images, labels, _ = data

            ### From numpy to torch variables
            labels = to_var(labels)
            # labels = torch.argmax(labels, dim=1)
            images = to_var(images)

            ################### Train ###################
            #-- The CNN makes its predictions (forward pass)
            net_predictions = net.forward(images)

            #-- Compute the losses --#
            # THIS FUNCTION IS TO CONVERT LABELS TO A FORMAT TO BE USED IN THIS CODE
            segmentation_classes = getTargetSegmentation(labels)
            
            # COMPUTE THE LOSS
            CE_loss_value = CE_loss(net_predictions, segmentation_classes) # XXXXXX and YYYYYYY are your inputs for the CE
            lossTotal = CE_loss_value
            pred = softMax(net_predictions)
            # masks = torch.argmax(pred, dim=1)
            # plt.imshow(masks[3])
            # plt.colorbar()
            # plt.show()

            # DO THE STEPS FOR BACKPROP (two things to be done in pytorch)
            lossTotal.backward()
            optimizer.step()

            confmat = ConfusionMatrix(task="multiclass", num_classes=4)
            confmat = confmat(net_predictions, segmentation_classes).numpy()
            accuracy = np.array([confmat[0,0]/confmat[:,0].sum(),
                        confmat[1,1]/confmat[:,1].sum(),
                        confmat[2,2]/confmat[:,2].sum(),
                        confmat[3,3]/confmat[:,3].sum(),]).astype(float)

            accuracy[accuracy==float('nan')] = 0
            mean_acc += accuracy
            n += 1
            
            # THIS IS JUST TO VISUALIZE THE TRAINING 
            # DSCEpoch.append(computeDSC(net_predictions, segmentation_classes))
            lossEpoch.append(lossTotal.cpu().data.numpy())
            printProgressBar(j + 1, num_batches,
                             prefix="[Training] Epoch: {} ".format(i),
                             length=15,
                             suffix=" Loss: {:.4f}, Acc: [{:.4f},{:.4f},{:.4f},{:.4f}]".format(lossTotal,accuracy[0],accuracy[1],accuracy[2],accuracy[3]))
        
        # DSCEpoch = np.asarray(DSCEpoch).mean()
        # print(DSCEpoch)
        mean_acc = mean_acc / n

        lossEpoch = np.asarray(lossEpoch)
        lossEpoch = lossEpoch.mean()

        lossTotalTraining.append(lossEpoch)

        
        printProgressBar(num_batches, num_batches,
                             done="[Training] Epoch: {}, LossG: {:.4f}, Acc: [{:.4f},{:.4f},{:.4f},{:.4f}]".format(i,lossEpoch,mean_acc[0],mean_acc[1],mean_acc[2],mean_acc[3]))
        net.eval()
        n = 0
        for j, data_val in enumerate(val_loader):
            images_val, labels_val, _ = data_val
            labels_val = to_var(labels_val)
            images_val = to_var(labels_val)
            
            net_predictions_val = net.forward(images_val.float())

            segmentation_classes_val = getTargetSegmentation(labels_val)

            CE_loss_value_val = CE_loss(net_predictions_val, segmentation_classes_val) # XXXXXX and YYYYYYY are your inputs for the CE

            confmat_val = ConfusionMatrix(task="multiclass", num_classes=4)
            confmat_val = confmat_val(net_predictions_val, segmentation_classes_val).numpy()
            accuracy_val = np.array([confmat_val[0,0]/confmat_val[:,0].sum(),
                            confmat_val[1,1]/confmat_val[:,1].sum(),
                            confmat_val[2,2]/confmat_val[:,2].sum(),
                            confmat_val[3,3]/confmat_val[:,3].sum(),]).astype(float)

            accuracy_val[accuracy_val==float('nan')] = 0
            mean_val_acc += accuracy_val
            n += 1

            lossValEpoch.append(CE_loss_value_val.cpu().data.numpy())
            printProgressBar(j + 1, num_batches_val,
                             prefix="[Validation] Epoch: {} ".format(i),
                             length=15,
                             suffix=" Loss: {:.4f}, Acc: [{:.4f},{:.4f},{:.4f},{:.4f}] ".format(CE_loss_value_val,accuracy_val[0],accuracy_val[1],accuracy_val[2],accuracy_val[3]))
        
        mean_val_acc = mean_val_acc / n

        lossValEpoch = np.asarray(lossValEpoch)
        lossValEpoch = lossValEpoch.mean()

        if lossValEpoch < Best_loss_val:
            Best_loss_val = lossValEpoch
            BestEpoch = i
            if not os.path.exists('./models/' + modelName):
                os.makedirs('./models/' + modelName)
            torch.save({'epoch': epoch,
                        'batch_size':batch_size,
                        'batch_size_val':batch_size_val,
                        'lr':lr,
                        'model_state_dict': net.state_dict(),
                        'optimizer_state_dict': optimizer.state_dict(),
                        'train_loss': lossEpoch,
                        'val_loss': lossValEpoch,
                        }, './models/' + modelName + '/best_model')
            np.save(os.path.join(directory, 'Losses.npy'), lossTotalTraining)


        printProgressBar(num_batches_val, num_batches_val,
                             done="[Validation] Epoch: {}, LossG: {:.4f}, Acc: [{:.4f},{:.4f},{:.4f},{:.4f}]".format(i,lossValEpoch,mean_val_acc[0],mean_val_acc[1],mean_val_acc[2],mean_val_acc[3]))
        
def LoadTraining(modelName, n_epochs):
    if os.path.exists('./models/'+modelName+'/best_model'):
        checkpoint = torch.load('./models/'+modelName+'/best_model')
    else :
        batch_size = 16
        batch_size_val = 8
        lr = 0.001
        checkpoint = {
            'epoch': 0,
            'batch_size': batch_size,
            'batch_size_val':batch_size_val,
            'lr':lr,
            'model_state_dict': None,
            'optimizer_state_dict': None,
            'train_loss': 0,
            'val_loss': 1000,
            }
    
    runTraining(modelName, checkpoint, n_epochs)

In [10]:
runTraining('Test_model')

----------------------------------------
~~~~~~~~  Starting the training... ~~~~~~
----------------------------------------
 Dataset: ./Data/ 
~~~~~~~~~~~ Creating the UNet model ~~~~~~~~~~
 Model Name: Test_Model
Total params: 60,664
~~~~~~~~~~~ Starting the training ~~~~~~~~~~
[Training] Epoch: 0 [DONE]                                                                   
[Training] Epoch: 0, LossG: 1.8645, Acc: [0.9291,0.0082,0.0088,0.0110]                                       
[Validation] Epoch: 0 [DONE]                                                                    
[Validation] Epoch: 0, LossG: 1.6057, Acc: [0.1078,0.2711,0.0021,0.0000]                                     
[Training] Epoch: 1 [DONE]                                                                   
[Training] Epoch: 1, LossG: 1.5915, Acc: [0.9329,0.0066,0.0070,0.0105]                                       
[Validation] Epoch: 1 [DONE]                                                                    
[Validat

KeyboardInterrupt: 

In [ ]:

transform = transforms.Compose([
    transforms.ToTensor()
])

mask_transform = transforms.Compose([
    transforms.ToTensor()
])

train_set_full = medicalDataLoader.MedicalImageDataset('train',
                                                    'Data/',
                                                    transform=transform,
                                                    mask_transform=mask_transform,
                                                    augment=False,
                                                    equalize=False)

train_loader_full = DataLoader(train_set_full,
                            batch_size=16,
                            worker_init_fn=np.random.seed(0),
                            num_workers=0,
                            shuffle=True)

print(len(train_set_full))
for item in train_set_full:
    print(len(item))

NameError: name 'transforms' is not defined